In [226]:
import pandas as pd
import numpy as np
from fuzzywuzzy import process
from IPython.display import clear_output
from time import sleep

In [151]:
new_scholar_works = pd.read_csv('new_scholar_works.csv')

In [152]:
new_scholar_works.sample(25)

,authors,collaborators,titles,dates,abstracts
10081,"Stout, Jeffrey R.","Aragon, Alan A.; Schoenfeld, Brad J.; Wildman,...",International Society Of Sports Nutrition Posi...,2017,Position Statement: The International Society ...
4877,"Huynh, Steven","Pascoe, Ben; Méric, Guillaume; Yahara, Koji; W...",Local Genes For Local Bacteria: Evidence Of Al...,2017,The genetic structure of bacterial populations...
2119,"Conlee, R.K.","Nelson, Arnold; Arnall, D.A.; Loy, S.F.; Silve...",Consequences of Combining Strength and Enduran...,1990,A common belief among many clinicians and trai...
205,"Almassian, Bijan","Lee, King C.; Zheng, Li-Mou; Luo, Xiang; Clair...",Comparative evaluation of the acute toxic effe...,2000,The objective of these studies was to perform ...
2007,"Cloninger, Cheri","Cloninger, Cheri; Felton, Marilyn; Paul, Bonni...",Control of pH during plasmid preparation by al...,2008,Alkaline lysis of Escherichia coli is usually ...
585,"Balmaceda, Laura A.","Muñoz-Jaramillo, Andrés; Senkpeil, Ryan R.; Wi...",Small-Scale And Global Dynamos And The Area An...,2015,"In this work, we take advantage of 11 differen..."
2131,"Conroy, Lauren","Conroy, Lauren; Gray, David","Male Armaments And Reproductive Behavior In ""N...",2015,Males of many species possess striking weaponr...
11874,"Yule, Doug","Meigs, Andrew; Yule, Doug; Blythe, Ann; Burban...",Implications Of Distributed Crustal Deformatio...,2003,Spatial and temporal patterns of exhumation ar...
901,"Bernthal, Nicholas M.","Bernthal, Nicholas M.; Schwartz, Adam J.; Oake...",How Long Do Endoprosthetic Reconstructions For...,2010,Background As the life expectancy of patients ...
1557,"Case, David A.","Joung, In Suk; Luchko, Tyler; Case, David A.",Simple electrolyte solutions: Comparison of DR...,2013,Using the dielectrically consistent reference ...


In [153]:
authors = new_scholar_works['authors'].tolist()

In [154]:
len(authors)

12199

### full_name_parser function

This function will clean up full names found in documents, attempting to seperate their full names into first, middle, and last into a dictionary.

The only draw back is asian names who seem say their last names first followed by the first names.

It mostly works okay. It doesn't always split the first and middle names correctly but it seems to always seperate out the last name which for this situation works great because we are trying to do a fuzzy match on the last names with other names that we have in other files.

In [162]:
def full_name_parser(full_name):

    d = {
        'first' : '',
        'middle' : '',
        'last' : ''
    }
    
    # check by comma space
    if ', ' in full_name:
        split_by_comma_space = full_name.split(', ')
        d['last'] = split_by_comma_space[0]
        split_by_space = split_by_comma_space[1].split(' ')
        if len(split_by_space) == 1:
            d['first'] = split_by_space[0]
        elif len(split_by_space) == 2:
            d['first'] = split_by_space[0]
            d['middle'] = split_by_space[1]
            
    elif len(full_name.split()) == 1:
        
        d['last'] = full_name.strip()
        
    elif len(full_name.split()) == 2:
        
        split_by_space = full_name.split()
        d['first'] = split_by_space[0]
        d['last'] = split_by_space[1]
        
    elif len(full_name.split()) == 3:
        
        split_by_space = full_name.split()
        d['first'] = split_by_space[0]
        d['middle'] = split_by_space[1]
        d['last'] = split_by_space[2]
        
    elif len(full_name.split()) == 4:
        split_by_space = full_name.split()
        d['first'] = split_by_space[0]
        d['middle'] = split_by_space[1]
        d['last'] = split_by_space[2] + " " + split_by_space[3]
        
    return d


Here I actually use the function to build a list of dictionaries.

In [156]:
full_names=[]
for i, full_name in enumerate(authors):
    full_names.append(full_name_parser(full_name))
        

Now I'll create parallel lists out of the dictionaries

In [157]:
first = []
middle = []
last = []
for d in full_names:
    first.append(d['first'])
    middle.append(d['middle'])
    last.append(d['last'])

I'll add the lists as columns to new_scholar_works

In [158]:
new_scholar_works['first_names'] = first
new_scholar_works['middle_names'] = middle
new_scholar_works['last_names'] = last

In [163]:
new_scholar_works.head()

,authors,collaborators,titles,dates,abstracts,first_names,middle_names,last_names
0,"A'Hearn, M.F.","Meech, K.J.; A'Hearn, M.F.; Adams, J.A.; Bacci...",EPOXI: comet 103P/Hartley 2 observations from ...,2011,Earth- and space-based observations provide sy...,M.F.,,A'Hearn
1,A.C. Asperti,A.C. Asperti; H.P. De Castro; M.H. Noronha; As...,Compact homogeneous Einstein manifolds in codi...,1996,In this paper we study compact Riemannian homo...,A.C.,,Asperti
2,"Aballe, L.","Clarkson, J. D.; Fina, I.; Liu, Z. Q.; Lee, Y....",Hidden Magnetic States Emergent Under Electric...,2017,The ability to control a magnetic phase with a...,L.,,Aballe
3,"Abasi, Lannah","Snoussi, Mehdi; Talledo, Paul; Del Rosario, Na...",Population Dynamics of Antimicrobial Peptide's...,2017,Antimicrobial peptides (AMPs) are broad spectr...,Lannah,,Abasi
4,"Abdala, Caroline","Abdala, Caroline; Visser-Dumont, Leslie",Distortion Product Otoacoustic Emissions: A To...,2001,Distortion product otoacoustic emissions (DPOA...,Caroline,,Abdala


Now I can get just the last names so I can try to fuzzy match them to the names that we already have in other files.

I think I want to get the names that we already matched all_faculty and sandbox scraper so I can get the email that's associated with them.

I might want to enumarate them. I just want a name that I can use to join with to attach emails to them.

What I can also do is attempt to build the names in a more proper format.

Find all the unique names.

Then try to do fuzzy matching on that.

In [189]:
first = new_scholar_works.first_names.tolist()
middle = new_scholar_works.middle_names.tolist()
last = new_scholar_works.last_names.tolist()

In [195]:
full_names = []
for i in range(len(first)):
    full_name = ""
    
    if first[i] != '':
        full_name += first[i]
    
    if first[i] == '':
        full_name+= middle[i]
    else:
        full_name += ' ' + middle[i]
        
    full_name += ' ' + last[i]
    
    full_names.append(full_name)

In [197]:
full_names[:10]

["M.F. A'Hearn",
 'A.C. Asperti',
 'L. Aballe',
 'Lannah Abasi',
 'Caroline Abdala',
 ' Abdeljawad',
 'Ashraf Abdelkerim',
 'J. Abdulla',
 'T. Abe',
 'T. Abe']

In [231]:
uniques = np.unique(full_names)

In [232]:
len(uniques)

7221

In [198]:
new_final_matches_outer = pd.read_csv('new_final_matches_outer.csv')

In [199]:
new_final_matches_outer.head()

,index1,faculty,index2,faculty1,email,sandbox
0,0.0,Keji Chen,518.0,Keji Chen,keji.chen@csun.edu,https://www.sandbox.csun.edu/faculty/keji.chen
1,1.0,Michael E. Doron,772.0,Michael E Doron,michael.doron@csun.edu,https://www.sandbox.csun.edu/faculty/michael.d...
2,2.0,Rafi Efrat,826.0,Rafi Efrat,rafael.efrat@csun.edu,https://www.sandbox.csun.edu/faculty/rafael.efrat
3,3.0,Monica Gianni,1079.0,Monica Gianni,monica.gianni@csun.edu,https://www.sandbox.csun.edu/faculty/monica.gi...
4,4.0,Young-Won Her,1315.0,Young-won Her,youngwon.her@csun.edu,https://www.sandbox.csun.edu/faculty/youngwon.her


In [205]:
faculty1 = new_final_matches_outer.faculty1.tolist()

In [201]:
def get_matches(query,choices,limit=4):
    results = process.extract(query,[choices],limit=limit)
    return results

In [227]:
def fuzzy_list_cross_reference_list(faculty1,faculty2):
    
    indeces = []
    
    matched = 0
    
    for i, name in enumerate(faculty1):
        clear_output()
        print('{:.2%}'.format(i+1/len(faculty1)))
        print(matched/len(faculty1), "matches!")

        for j, name2 in enumerate(faculty2):

            if get_matches(name,name2)[0][1] > 87:

                indeces.append([i,name,j,name2])

                break
                
    return indeces

Here is an attempt at fuzzy matching using whole names from final_matches and whole names from new scholar works.

In [233]:
matches = fuzzy_list_cross_reference_list(faculty1,uniques)

99.98%


In [235]:
len(matches)

690

In [236]:
matches

[[0, 'Keji Chen', 10, ' Chen'],
 [2, 'Rafi Efrat', 5467, 'Rafi Efrat'],
 [7, 'David Liu', 1543, 'David Liu'],
 [8, 'Dat Dao Nguyen', 59, ' Uy'],
 [9, 'Rishma R Vedd', 5595, 'Rishma Vedd'],
 [10, 'Sung wook Yoon', 26, ' K.Y.'],
 [34, 'Raymond Chen', 10, ' Chen'],
 [36, 'Donna Driscoll', 1714, 'Donna Driscoll'],
 [37, 'Glen L Gray', 2294, 'Glen L. Gray'],
 [38, 'Christopher G Jones', 1198, 'Christopher G. Jones'],
 [42, 'James H Macklin', 30, ' Lin'],
 [45, 'Earl J Weiss', 62, ' Wei'],
 [47, 'Wei Cao', 62, ' Wei'],
 [49, 'Jongeun Kim', 3307, 'Jongeun Kim'],
 [53, 'Linda A Bradley', 30, ' Lin'],
 [54, 'Yoko Mimura', 7072, 'Yoko Mimura'],
 [64, 'Dena Herman-Mendes', 1611, 'Dena Herman'],
 [90, 'Hsin-hui Chiu', 2541, 'Hsin-Hui Chiu'],
 [91, 'James Chong', 2867, 'James Chong'],
 [92, 'Vicentiu M Covrig', 6687, 'Vicentiu Covrig'],
 [93, 'James P Dow', 2904, 'James P. Dow'],
 [94, 'Yanbo Jin', 7004, 'Yanbo Jin'],
 [95, 'Michael Phillips', 2152, 'G. Michael Phillips'],
 [99, 'Chao Chen', 10, ' 

Now I'll try just using the last names. I expect more matches but also more false positives.

I could also attempt to match last name and first name at the same time but seperately

In [ ]:
Kerry satamaro female
chris kechityan
ryan holbreg enterprenuarial
tim timmen icore the art of the pitch guy kawazaki 302010